# Vehicle Detection Project

In [2]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
from lesson_functions import *
import glob
import time

In [18]:
# create vehicle img list. We mark whether the data belongs to a certain GTI data directory.

vehicle_GTI_dirs = ['./data/vehicles/GTI_Far/', './data/vehicles/GTI_Left/', './data/vehicles/GTI_MiddleClose/', './data/vehicles/GTI_Right/']
vehicle_img_list = []
GTI_group_ids = []
cgroup_id = 1
for GTI_dir in vehicle_GTI_dirs:
    GTI_current_list = glob.glob(GTI_dir + '*.png')  
    vehicle_img_list = vehicle_img_list + GTI_current_list   
    GTI_group_ids = GTI_group_ids + [np.ones(len(GTI_current_list))*cgroup_id]
    cgroup_id += 1
KTTI_list = glob.glob('./data/vehicles/KITTI_extracted/*.png')
vehicle_img_list = vehicle_img_list+ KTTI_list
vehicle_group_id = np.concatenate(GTI_group_ids + [np.zeros(len(KTTI_list))])

In [16]:
# create vehicle img list and group ids for non-vehicle data
# note that GTI image for non-vehicle does not have constant width name, therefore we need to sort it.
non_vehicle_img_list = glob.glob('./data/non-vehicles/GTI/*.png') + glob.glob('./data/non-vehicles/Extras/*.png')
non_vehicle_group_id = np.zeros(len(non_vehicle_img_list))

In [6]:
vehicle_images =  [cv2_readRGB(w) for w in vehicle_img_list]
non_vehicle_images =  [cv2_readRGB(w) for w in non_vehicle_img_list]


In [7]:
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

In [14]:
color_space = 'HLS' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 8  # HOG orientations
pix_per_cell = 8 # HOG pixels per cell
cell_per_block = 2 # HOG cells per block
hog_channel = 'ALL' # Can be 0, 1, 2, or "ALL"
spatial_size = (16, 16) # Spatial binning dimensions
hist_bins = 16    # Number of histogram bins
spatial_feat = False # Spatial features on or off
hist_feat = False # Histogram features on or off
hog_feat = True # HOG features on or off
y_start_stop = [400, 700] # Min and max in y to search in slide_window()

In [8]:
x=set(range(0,5))

In [11]:
x.difference(set([2,3]))

{0, 1, 4}

In [34]:
def train_test_split_with_GTI(X, y, group_id, test_size, random_state):
    X_train_list = []
    y_train_list = []
    X_test_list = []
    y_test_list = []
    
    for c_id in np.unique(group_id):
        X_group = X[group_id == c_id,:]
        y_group = y[group_id == c_id]
        if c_id ==0:
            # If this is KTTI data then, split it 
            X_train, X_test, y_train, y_test = train_test_split(
                X_group, y_group, test_size=0.2, random_state=0, stratify=y_group)
        else:         
            np.random.seed(c_id)
            test_index = np.mod(np.random.randint(0, X_group.shape[0]) + np.arange(0, int(X_group.shape[0] * test_size)),
                                 X_group.shape[0])
            train_index = list(set(range(0, X_group.shape[0])).difference(set(test_index)))
            X_train = X_group[train_index,:]
            X_test = X_group[test_index,:]
            y_train = y_group[train_index]
            y_test = y_group[test_index]
            
        X_train_list.append(X_train)
        y_train_list.append(y_train)
        X_test_list.append(X_test)
        y_test_list.append(y_test)
    return np.concatenate(X_train_list, axis=0), np.concatenate(X_test_list, axis=0), np.concatenate(y_train_list), np.concatenate(y_test_list)

# experiment with HOG parameters

In [15]:
car_features = extract_features(vehicle_images, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)
notcar_features = extract_features(non_vehicle_images, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)
X = np.vstack((car_features, notcar_features)).astype(np.float64)                        
X_scaler = StandardScaler().fit(X)
scaled_X = X_scaler.transform(X)
y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))

C:\App\conda\envs\carnd-term1\lib\site-packages\skimage\feature\_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


In [20]:
group_id = np.concatenate([vehicle_group_id, non_vehicle_group_id])

In [25]:
X_train, X_test, y_train, y_test = train_test_split_with_GTI(scaled_X, y, group_id, test_size=0.2, random_state=0)

In [27]:
X_train.shape

(14211, 4704)

In [29]:
# Check the training time for the SVC
clf = LinearSVC(penalty='l1', dual=False)
t=time.time()
clf.fit(X_train, y_train)
t2 = time.time()

In [31]:
print(round(t2-t, 2), 'Seconds to train classifier...')
# Check the score of the SVC
print('Train Accuracy of Classifier = ', round(clf.score(X_train, y_train), 4))
print('Test Accuracy of Classifier = ', round(clf.score(X_test, y_test), 4))
# Check the prediction time for a single sample

99.67 Seconds to train classifier...
Train Accuracy of Classifier =  1.0
Test Accuracy of Classifier =  0.9828


In [41]:
group_id = np.concatenate([vehicle_group_id, non_vehicle_group_id]).astype(np.int64)
color_space_choice = ['RGB','HSV','LUV','HLS','YUV','YCrCb']
channel_choice = [0, 1, 2]

In [ ]:
result = {}

for color_space in color_space_choice:
    for hog_channel in channel_choice:
        print((color_space, hog_channel))
        car_features = extract_features(vehicle_images, color_space=color_space, 
                                spatial_size=spatial_size, hist_bins=hist_bins, 
                                orient=orient, pix_per_cell=pix_per_cell, 
                                cell_per_block=cell_per_block, 
                                hog_channel=hog_channel, spatial_feat=spatial_feat, 
                                hist_feat=hist_feat, hog_feat=hog_feat)
        notcar_features = extract_features(non_vehicle_images, color_space=color_space, 
                                spatial_size=spatial_size, hist_bins=hist_bins, 
                                orient=orient, pix_per_cell=pix_per_cell, 
                                cell_per_block=cell_per_block, 
                                hog_channel=hog_channel, spatial_feat=spatial_feat, 
                                hist_feat=hist_feat, hog_feat=hog_feat)
        X = np.vstack((car_features, notcar_features)).astype(np.float64)                        
        X_scaler = StandardScaler().fit(X)
        scaled_X = X_scaler.transform(X)
        y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))
        X_train, X_test, y_train, y_test = train_test_split_with_GTI(scaled_X, y, group_id, test_size=0.2, random_state=0)
        # Check the training time for the SVC
        clf = LinearSVC(penalty='l1', dual=False)
        t=time.time()
        clf.fit(X_train, y_train)
        t2 = time.time()
        print(round(t2-t, 2), 'Seconds to train classifier...')
        # Check the score of the SVC
        print('Train Accuracy of Classifier = ', round(clf.score(X_train, y_train), 4))
        print('Test Accuracy of Classifier = ', round(clf.score(X_test, y_test), 4))
        # Check the prediction time for a single sample
        result[(color_space, hog_channel)] = {'train': clf.score(X_train, y_train),
                                              'test': clf.score(X_test, y_test),
                                              'clf': clf}
        

('RGB', 0)


C:\App\conda\envs\carnd-term1\lib\site-packages\skimage\feature\_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


130.74 Seconds to train classifier...
Train Accuracy of Classifier =  0.9995
Test Accuracy of Classifier =  0.9327
('RGB', 1)
138.62 Seconds to train classifier...
Train Accuracy of Classifier =  0.9999
Test Accuracy of Classifier =  0.9391
('RGB', 2)
141.32 Seconds to train classifier...
Train Accuracy of Classifier =  0.9998
Test Accuracy of Classifier =  0.9383
('HSV', 0)
181.3 Seconds to train classifier...
Train Accuracy of Classifier =  0.9839
Test Accuracy of Classifier =  0.9132
('HSV', 1)
185.1 Seconds to train classifier...
Train Accuracy of Classifier =  0.9858
Test Accuracy of Classifier =  0.9132
('HSV', 2)
142.4 Seconds to train classifier...
Train Accuracy of Classifier =  0.9999
Test Accuracy of Classifier =  0.9383
('LUV', 0)
139.07 Seconds to train classifier...
Train Accuracy of Classifier =  1.0
Test Accuracy of Classifier =  0.9383
('LUV', 1)
158.71 Seconds to train classifier...
Train Accuracy of Classifier =  0.9886
Test Accuracy of Classifier =  0.9346
('LUV', 2

In [ ]:
y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))

In [ ]:
# Split up data into randomized training and test sets
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(
    scaled_X, y, test_size=0.2, random_state=0)

In [ ]:
scaled_X.shape

In [ ]:
# Use a linear SVC 


In [ ]:
# Check the training time for the SVC
t=time.time()
clf.fit(X_train, y_train)
t2 = time.time()

In [ ]:
image = cv2_readRGB('./test_images/test1.jpg')
plt.figure()
plt.imshow(image)

In [ ]:
windows

## Set windows

In [ ]:
windows1 = slide_window(image, x_start_stop=[None, None], y_start_stop=[400, 550], 
                         xy_window=(64, 64), xy_overlap=(0.5, 0.5))
windows2 =slide_window(image, x_start_stop=[None, None], y_start_stop=[464, 600], 
                       xy_window=(96, 96), xy_overlap=(0.5, 0.5))
windows3 =slide_window(image, x_start_stop=[None, None], y_start_stop=[496, 700], 
                       xy_window=(128, 128), xy_overlap=(0.5, 0.5))

windows = windows1 + windows2 + windows3

In [ ]:
test_images = glob.glob('./test_images/*.jpg')
for test_image in test_images:
    draw_image = cv2_readRGB(test_image) 
    window_img = draw_boxes(draw_image, windows1, color=(0, 0, 255), thick=6)                    
    window_img = draw_boxes(window_img, windows2, color=(0, 255, 0), thick=6)
    window_img = draw_boxes(window_img, windows3, color=(255, 0, 0), thick=6) 
    plt.figure(figsize=(20,8))
    plt.imshow(window_img)
    plt.title(test_image)

In [ ]:
test_images = glob.glob('./test_images/*.jpg')
for test_image in test_images:
    image = cv2_readRGB(test_image) 
    draw_image = np.copy(image)

    hot_windows = search_windows(image, windows, clf, X_scaler, color_space=color_space, 
                            spatial_size=spatial_size, hist_bins=hist_bins, 
                            orient=orient, pix_per_cell=pix_per_cell, 
                            cell_per_block=cell_per_block, 
                            hog_channel=hog_channel, spatial_feat=spatial_feat, 
                            hist_feat=hist_feat, hog_feat=hog_feat)                       

    window_img = draw_boxes(draw_image, hot_windows, color=(0, 0, 255), thick=6)                    
    plt.figure(figsize=(20,8))
    plt.imshow(window_img)
    plt.title(test_image)


In [ ]:
# test on GTI

In [ ]:
vehicle_GTI_dirs = ['./data/vehicles/GTI_Far/', './data/vehicles/GTI_Left/', './data/vehicles/GTI_MiddleClose/', './data/vehicles/GTI_Right/']
vehicle_GTI_list = []
for GTI_dir in vehicle_GTI_dirs:
    vehicle_GTI_list = vehicle_GTI_list + glob.glob(GTI_dir + '*.png')  

In [ ]:
GTI_images =  [cv2_readRGB(w) for w in vehicle_GTI_list]
                     


In [ ]:
car_GTI_features = extract_features(GTI_images, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)

In [ ]:
X_GTI = np.vstack(car_GTI_features).astype(np.float64)   

In [ ]:
np.mean(clf.predict(X_scaler.transform(X_GTI)))

In [ ]:
s=np.where(clf.predict(X_scaler.transform(X_GTI))==0)[0]

In [ ]:
for u in s:
    plt.figure()
    plt.imshow(GTI_images[u])